# Analyze MCTruth Example

OK, now that we have that out of the way, let's do a little work with art-ROOT files via gallery in a ROOT C++-kernel notebook.

Remember, `gallery` is the set of utilities/functions that allow us to have a sort of "easy access" to art-ROOT files, which are the standard output format for our simulation and reconstruction. You can read more about gallery, and see some additional examples from the [art group's webpage](http://art.fnal.gov/gallery/).

---
## Finding data objects

This is not a full art/gallery demonstration, so I'm going to assume that you are reasonably aware of how to use and navigate the data format. One of the most useful things though is to know where to see the data objects. In general, when you setup a ups product, the source code is available to peruse in the `$THATPRODUCT_DIR/source` area. So, important areas are:
* `$LARDATAOBJ_DIR/source/lardataobj`
* `$NUSIMDATA_DIR/source/nusimdata`

The contents of those areas are here:

In [1]:
.! (cd $LARDATAOBJ_DIR/source/ && ls lardataobj/*/*.h)

lardataobj/AnalysisBase/BackTrackerMatchingData.h
lardataobj/AnalysisBase/Calorimetry.h
lardataobj/AnalysisBase/CosmicTag.h
lardataobj/AnalysisBase/FlashMatch.h
lardataobj/AnalysisBase/MVAOutput.h
lardataobj/AnalysisBase/MVAPIDResult.h
lardataobj/AnalysisBase/ParticleID.h
lardataobj/AnalysisBase/T0.h
lardataobj/AnalysisBase/classes.h
lardataobj/MCBase/MCBaseException.h
lardataobj/MCBase/MCDataHolder.h
lardataobj/MCBase/MCHit.h
lardataobj/MCBase/MCHitCollection.h
lardataobj/MCBase/MCLimits.h
lardataobj/MCBase/MCShower.h
lardataobj/MCBase/MCStep.h
lardataobj/MCBase/MCTrack.h
lardataobj/MCBase/MCWire.h
lardataobj/MCBase/MCWireCollection.h
lardataobj/MCBase/classes.h
lardataobj/OpticalDetectorData/ChannelData.h
lardataobj/OpticalDetectorData/ChannelDataGroup.h
lardataobj/OpticalDetectorData/FIFOChannel.h
lardataobj/OpticalDetectorData/OpticalRawDigit.h
lardataobj/OpticalDetectorData/OpticalTypes.h
lardataobj/OpticalDetectorData/PMTTrigger.h
lardataobj/OpticalDetectorData/classes.h
lardatao

In [2]:
.! (cd $NUSIMDATA_DIR/source/ && ls nusimdata/*/*.h)

nusimdata/SimulationBase/GTruth.h
nusimdata/SimulationBase/MCFlux.h
nusimdata/SimulationBase/MCNeutrino.h
nusimdata/SimulationBase/MCParticle.h
nusimdata/SimulationBase/MCTrajectory.h
nusimdata/SimulationBase/MCTruth.h
nusimdata/SimulationBase/classes.h
nusimdata/SimulationBase/simb.h


---
## Necessary includes and linking libraries <a href="includes"></a>

There are a number of neccesary `include` statements to use gallery, and you'll also need to include headers for the objects you're going to want to analyze from the event. You can do a simple include statement directly in a cell in the macro:

```
//gallery includes
#include "canvas/Utilities/InputTag.h"
#include "gallery/Event.h"
#include "gallery/ValidHandle.h"
#include "canvas/Persistency/Common/FindMany.h"
#include "canvas/Persistency/Common/FindOne.h"
#include "canvas/Persistency/Common/FindManyP.h"
#include "canvas/Persistency/Common/FindOneP.h"
```

That may often work, but I sometimes have trouble, especially when it comes to getting proper templates known by ROOT for associations. Instead, I find doing things in the following way rather useful.

First, I setup a directory to contain a lot of "common" use files. These are collections of headers and maybe other functions/utilities I think I might use a lot. You can see that directory <a href="common/">here</a>. For this example in particular, we're going to load in these two files:
* <a href="common/gallery_includes.h">common/gallery_includes.h</a>
* <a href="common/larsoftobj_includes.h">common/larsoftobj_includes.h</a>

Now, when we load these in, we're going to tell ROOT to compile them (with an optimization flag!). ROOT is smart enough to know if there's been a change since the last compilation, so it will only do it as necessary. We use the `ProcessLine()` functionality that you can use just the same from the ROOT command-line:

In [3]:
gROOT->ProcessLine(".L common/gallery_includes.h+O");
gROOT->ProcessLine(".L common/larsoftobj_includes.h+O");

And, just like that, we should be good to go on to writing code!

---
## Declarations

OK, remeber what we talked about in the <a href="ROOT_CPP_Kernel_Example.ipynb">C++-kernel tutorial</a>, where it makes sense to declare the variables we're going to be using in cells ahead of where we use them. Let's go ahead and do that here too.

Note: this is a process of trial and error, of course. You're not going to perfectly set everything up in one go before you even write the bulk of your code doing the analysis on each event. Just remember a few things:
- You can "Restart & Run All" for the kernel
- You can execute single cells to declare something new, and then re-execute a cell where you've made a change using the thing in that cell.
- You don't have to execute cells in order. And you can move cells around as you wish.

That said, let's talk about what we'll do here, and then what we want to set up. Let's say we want to loop over the MCTruth objects in a file and store the neutrino and lepton (if charged current) kinematics in a tree, and then write that tree to a file, and maybe make a plot or two in this notebook too.

So, let's list out what we'll need:
- a vector of strings to hold an input file list;
- an `art::InputTag` to specify the label for the MCTruth object in the event;
- the output tree;
- an output file; and,
- a canvas.

Let's even think ahead slightly and give ourselves a "verbose" flag that we can turn on and off for testing/debugging.


In [4]:
// string to contain our input files
std::vector<std::string> filenames;

//input tag for the generator
art::InputTag gen_tag;

//placeholder for the output file
TFile* f_output;

//a canvas in case we want one
TCanvas canvas("canvas","MyCanvas");

//setup the ROOT output tree
unsigned int run, event;
int nu_pdg,ccnc,mode;
float q2,nu_energy;
float vtx_x,vtx_y,vtx_z;
float lep_energy,lep_p,lep_px,lep_py,lep_pz;
  
TTree* fAnaTree = new TTree("ana","Nu Interaction Tree");
fAnaTree->Branch("run",&run,"run/I");
fAnaTree->Branch("event",&event,"event/I");
fAnaTree->Branch("nu_pdg",&nu_pdg,"nu_pdg/I");
fAnaTree->Branch("ccnc",&ccnc,"ccnc/I");
fAnaTree->Branch("mode",&mode,"mode/I");
fAnaTree->Branch("q2",&q2,"q2/F");
fAnaTree->Branch("nu_energy",&nu_energy,"nu_energy/F");
fAnaTree->Branch("vtx_x",&vtx_x,"vtx_x/F");
fAnaTree->Branch("vtx_y",&vtx_y,"vtx_y/F");
fAnaTree->Branch("vtx_z",&vtx_z,"vtx_z/F");
fAnaTree->Branch("lep_energy",&lep_energy,"lep_energy/F");
fAnaTree->Branch("lep_p",&lep_p,"lep_p/F");
fAnaTree->Branch("lep_px",&lep_px,"lep_px/F");
fAnaTree->Branch("lep_py",&lep_py,"lep_py/F");
fAnaTree->Branch("lep_pz",&lep_pz,"lep_pz/F");

//verbosity mode flag
bool verbose=false;

## Initializations

Right, now that everything has been declared, we actually want to initialize a lot of it. Note: this initialization cell should be able to be run repeatedly now, given that we're not re-declaring anythng, but just re-assigning things.

One quick trick: while you might imagine we don't need to really think about the output file until later, it's helpful to setup the output file early, as it gives ROOT a place to write things into temporarily. This is especially important if you run over a ton of events.

In [5]:
//setup our output file here to avoid write buffer problems
//and, you know, make sure you change this to something in your area :)

f_output = new TFile("/uboone/data/users/wketchum/tmp.root","RECREATE");

//setup our input file and input tag
//the nice thing about doing this on the gpvms is having easy access to all your data areas!
filenames = 
{
  "/uboone/data/users/wketchum/prodgenie_bnb_intrinsic_nue_uboone_0_20170413T202620_gen2_419cbfe3-cbf6-4a31-808b-ef9fbbae0254.root"
};

gen_tag = { "generator" };

---
## The gallery loop

With that all set, we are ready to run the gallery loop! Again, not gonna give you the full gallery demo here, but you can use this as a working example of how to do it.

In [9]:
verbose = true;
fAnaTree->Reset();
for(gallery::Event ev(filenames) ; !ev.atEnd(); ev.next()) {

    if(verbose){
        std::cout << "Processing " 
                  << "Run " << ev.eventAuxiliary().run() << ", "
                  << "Event " << ev.eventAuxiliary().event() << endl;
    }
    
    auto const& mctruth_handle = ev.getValidHandle<vector<simb::MCTruth>>(gen_tag);
    auto const& truth_vec(*mctruth_handle);
    if(verbose) std::cout << "\tThere are " << truth_vec.size() << " MCTruths in this event." << std::endl;

    //Loop over and print them out
    for( simb::MCTruth const& mct : truth_vec){
        
        //we only want to look at neutrinos...
        if(mct.Origin()!=simb::Origin_t::kBeamNeutrino) continue;
      
        //get the neutrino and start filling our output info
        auto const& nu = mct.GetNeutrino();
        ccnc = nu.CCNC();
        mode = nu.Mode();
        q2 = nu.QSqr();
        nu_energy = nu.Nu().E();
        nu_pdg = nu.Nu().PdgCode();
        vtx_x = nu.Nu().Vx();
        vtx_y = nu.Nu().Vy();
        vtx_z = nu.Nu().Vz();
      
      if(verbose) 
          std::cout << "\t\t" << nu_pdg << " " << nu_energy << " " << ccnc << " " << mode << " "
                    << nu.Nu().Vx() << "," << nu.Nu().Vy() << "," << nu.Nu().Vz() << std::endl;
      
      if(ccnc==0) { //CC
          lep_energy = nu.Lepton().E();
          lep_p = nu.Lepton().P();
          lep_px = nu.Lepton().Px();
          lep_py = nu.Lepton().Py();
          lep_pz = nu.Lepton().Pz();
      }
      else{
          lep_energy = -999;
          lep_p = -999;
          lep_px = -999;
          lep_py = -999;
          lep_pz = -999;
      }
      fAnaTree->Fill();

    }//end loop over truth object

}//end loop over events


Successfully opened file /uboone/data/users/wketchum/prodgenie_bnb_intrinsic_nue_uboone_0_20170413T202620_gen2_419cbfe3-cbf6-4a31-808b-ef9fbbae0254.root
Processing Run 2, Event 469951
	There are 1 MCTruths in this event.
		12 1.26009 0 1 149.549,20.2299,342.826
Processing Run 2, Event 469952
	There are 1 MCTruths in this event.
		12 1.72622 0 2 126.533,3.13313,903.273
Processing Run 2, Event 469953
	There are 1 MCTruths in this event.
		12 0.315355 0 0 103.413,63.6562,791.466
Processing Run 2, Event 469954
	There are 1 MCTruths in this event.
		12 0.726046 0 0 216.097,-88.0515,262.828
Processing Run 2, Event 469955
	There are 1 MCTruths in this event.
		12 2.51577 1 0 141.224,-13.5385,987.196
Processing Run 2, Event 469956
	There are 1 MCTruths in this event.
		12 1.35134 0 1 206.98,94.4059,961.351
Processing Run 2, Event 469957
	There are 1 MCTruths in this event.
		12 1.39069 0 0 97.6515,-21.6837,705.237
Processing Run 2, Event 469958
	There are 1 MCTruths in this event.
		12 1.33284

%MSG-d ProductID: 
Product created with id: [3130632973] from canonical product name: "simb::MCTruths_generator__GenieGen."
%MSG
%MSG-d ProductID: 
Product created with id: [4283826371] from canonical product name: "simb::MCTruths_generator__G4."
%MSG
%MSG-d ProductID: 
Product created with id: [852458230] from canonical product name: "simb::MCTruths_generator__Detsim."
%MSG


---
## Analyzing and storing the output

Since we've already set everything up, we're ready to just look at it all in this notebook if we'd like! Let's make a few quick plots, like ... CC vs. NC? And neutrino energy vs. lepton energy in CC interactions?

In [15]:
%jsroot on
canvas.cd();
fAnaTree->Draw("ccnc>>h");
h->SetMinimum(0);
canvas.Draw();

In [19]:
fAnaTree->SetMarkerStyle(8);
fAnaTree->Draw("nu_energy:lep_energy","ccnc==0");
canvas.Draw();

OK, and then, so we can save our work and analyze it in a notebook later, let's save the tree to our output file.

In [20]:
f_output->cd();
fAnaTree->Write();
f_output->Close();